<a href="https://colab.research.google.com/github/surajghuwalewala/CE888_Data_Science_and_Decision_Making/blob/master/Assignment2/CE888_DecMeg2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DecMeg2014 - Decoding the Human Brain




In [0]:
## Kaggle details
import os
os.environ['KAGGLE_USERNAME'] = "surajghuwalewala" # username from the json file
os.environ['KAGGLE_KEY'] = "c14ff4f2803c1ffb349c4b9e1a57020b" # key from the json file

In [0]:
DOWNLOAD_DATA = True

In [0]:
if DOWNLOAD_DATA:
    !kaggle competitions download -c decoding-the-human-brain # api copied from kaggle
    !unzip -q -n '/content/*.zip'  ## unzips all archives  q - quite, n - don't overwrite

random_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train_07_12.zip: Skipping, found more recently modified local copy (use --force to force download)
train_13_16.zip: Skipping, found more recently modified local copy (use --force to force download)
train_01_06.zip: Skipping, found more recently modified local copy (use --force to force download)
test_17_23.zip: Skipping, found more recently modified local copy (use --force to force download)

4 archives were successfully processed.


In [0]:
import numpy as np
import scipy.io as sio
# # Several libraries for designing the CNN
from tensorflow.keras.utils  import normalize, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, SeparableConv2D
from tensorflow.keras.layers import BatchNormalization, Flatten
from tensorflow.keras.layers import Dense, Activation, Dropout,  MaxPooling2D
from tensorflow.keras.optimizers import SGD as SGD_Loss

In [0]:
import glob

train_files = np.sort( glob.glob('/content/data/train*') )
test_files = np.sort( glob.glob('/content/data/test*') )

In [0]:
train_files

array(['/content/data/train_subject01.mat',
       '/content/data/train_subject02.mat',
       '/content/data/train_subject03.mat',
       '/content/data/train_subject04.mat',
       '/content/data/train_subject05.mat',
       '/content/data/train_subject06.mat',
       '/content/data/train_subject07.mat',
       '/content/data/train_subject08.mat',
       '/content/data/train_subject09.mat',
       '/content/data/train_subject10.mat',
       '/content/data/train_subject11.mat',
       '/content/data/train_subject12.mat',
       '/content/data/train_subject13.mat',
       '/content/data/train_subject14.mat',
       '/content/data/train_subject15.mat',
       '/content/data/train_subject16.mat'], dtype='<U33')

In [0]:
# ## merge data.

# def mergeData(datafiles, data_type = 'train'):
#     signal = np.array([])
#     label = np.array([])

#     for file in datafiles:
#         dat = sio.loadmat(file)
#         X = dat['X']
#         signal = np.vstack([signal,X]) if signal.size else X
        
#         if data_type == 'train':
#             y = dat['y']
#             label = np.vstack([label,y]) if label.size else y

#     return signal, label


In [0]:
def create_model(batch_norm = True):
    ## Defining the CNN model

    kernel_size = (5,5)

    ## Initialize the model
    model = Sequential()

    ##reading a file to get input shape
    train_X = sio.loadmat(train_files[0])['X']
    train_X = train_X.reshape(-1, train_X.shape[1], train_X.shape[2], 1)

    ## Layer 1
    model.add(Conv2D(4, kernel_size, padding = 'same', input_shape=train_X.shape[1:]))
    if batch_norm: model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if not batch_norm:model.add(Dropout(0.25))

    ## Layer 2
    model.add(SeparableConv2D(4, kernel_size, padding = 'same'))
    if batch_norm: model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if not batch_norm:model.add(Dropout(0.25))


    ## Layer 3
    model.add(SeparableConv2D(16, kernel_size, padding = 'same'))
    if batch_norm: model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if not batch_norm:model.add(Dropout(0.25))

    ## Layer 4
    model.add(SeparableConv2D(8, kernel_size, padding = 'same'))
    if batch_norm: model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if not batch_norm:model.add(Dropout(0.25))

    ## Layer 5
    model.add(SeparableConv2D(4, kernel_size, padding = 'same'))
    if batch_norm: model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if not batch_norm:model.add(Dropout(0.25))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    ## ------------ CLASSIFICATION ------------ ##

    ## Layer 6
    model.add(Dense(256))

    ## Layer 7
    model.add(Dense(128))

    ## Layer 8
    model.add(Dense(32))

    ## Layer 9
    model.add(Dense(1))
    model.add(Activation('sigmoid'))



    model.compile(loss= 'binary_crossentropy',
                optimizer = SGD_Loss(learning_rate=0.01, momentum=0.0, nesterov=True),
                metrics= ['accuracy'])
    
    return model

In [0]:
model = create_model(batch_norm=False)
norm_model = create_model(batch_norm=True)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 306, 375, 4)       104       
_________________________________________________________________
activation (Activation)      (None, 306, 375, 4)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 153, 187, 4)       0         
_________________________________________________________________
dropout (Dropout)            (None, 153, 187, 4)       0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 153, 187, 4)       120       
_________________________________________________________________
activation_1 (Activation)    (None, 153, 187, 4)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 76, 93, 4)         0

In [0]:
norm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 306, 375, 4)       104       
_________________________________________________________________
batch_normalization (BatchNo (None, 306, 375, 4)       16        
_________________________________________________________________
activation_6 (Activation)    (None, 306, 375, 4)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 153, 187, 4)       0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 153, 187, 4)       120       
_________________________________________________________________
batch_normalization_1 (Batch (None, 153, 187, 4)       16        
_________________________________________________________________
activation_7 (Activation)    (None, 153, 187, 4)      

In [0]:
from tensorflow.keras.utils import normalize


n_epochs = 20

for i,file in enumerate(train_files):
    
    print("\n-------------------------------------------------\n")
    print("Working on Subject {}".format(i))
    data = sio.loadmat(file)
    X = normalize(data['X'], axis=1)
    y  = data['y']
    
    # reshape for CNN
    X = X.reshape(-1, X.shape[1], X.shape[2], 1)

    print("\nWithout Batch Norm")
    model.fit(X,y, epochs=n_epochs, batch_size=25, validation_split=0.2)
    print("\nWith Batch Norm")
    norm_model.fit(X,y, epochs=n_epochs, batch_size=25, validation_split=0.2)

    


-------------------------------------------------

Working on Subject 0

Without Batch Norm
Epoch 1/20
19/19 [==============================] - 2s 98ms/step - loss: 0.6945 - accuracy: 0.4779 - val_loss: 0.6931 - val_accuracy: 0.5042
Epoch 2/20
19/19 [==============================] - 2s 83ms/step - loss: 0.6940 - accuracy: 0.4947 - val_loss: 0.6928 - val_accuracy: 0.5210
Epoch 3/20
19/19 [==============================] - 2s 83ms/step - loss: 0.6931 - accuracy: 0.4884 - val_loss: 0.6932 - val_accuracy: 0.4790
Epoch 4/20
19/19 [==============================] - 2s 83ms/step - loss: 0.6938 - accuracy: 0.5032 - val_loss: 0.6931 - val_accuracy: 0.4958
Epoch 5/20
19/19 [==============================] - 2s 84ms/step - loss: 0.6939 - accuracy: 0.4863 - val_loss: 0.6936 - val_accuracy: 0.4790
Epoch 6/20
19/19 [==============================] - 2s 83ms/step - loss: 0.6933 - accuracy: 0.5053 - val_loss: 0.6935 - val_accuracy: 0.4790
Epoch 7/20
19/19 [==============================] - 2s 83ms/s

## Testing


In [0]:
pred = np.array([])
norm_pred = np.array([])

Since testing has to be done in a perfect order to maintain the sequence of predictions for the submission, we read file by their specific name

In [0]:
for i,file in enumerate(test_files):
    print("Case {}".format(i+1))
    data = sio.loadmat("/content/data/test_subject{}.mat".format(17+i))  ## Very specific numbering for this case. (Works as of 12/04/2020)
    X = normalize(data['X'], axis=1)    
    # reshape for CNN
    X = normalize(X.reshape(-1, X.shape[1], X.shape[2], 1), axis=1)
    pred = np.vstack([pred, model.predict(X)]) if pred.size else model.predict(X)
    norm_pred = np.vstack([norm_pred, norm_model.predict(X)]) if norm_pred.size else norm_model.predict(X)



Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7


In [0]:
# test_X, _ = mergeData(test_files, 'test')

In [0]:
# hist = model.fit(train_X, train_y, epochs = 5, steps_per_epoch = 100, validation_split=0.2, shuffle=True, verbose=1)

In [0]:
## merging test data
# test_X, _ = mergeData(test_files, 'test')

In [0]:
# ## reshaping test
# test_X = normalize(test_X.reshape(-1, test_X.shape[1], test_X.shape[2], 1), axis=1)
# test_X.shape

In [0]:
# pred = model.predict(test_X)
# norm_pred = norm_model.predict(test_X)

In [0]:
# pred

In [0]:
# np.unique(pred)

In [0]:
predictions = np.floor(pred+0.5)
norm_predictions = np.floor(norm_pred+0.5)

In [0]:
predictions = np.array(predictions, dtype='int')
norm_predictions = np.array(norm_predictions, dtype='int')

In [0]:
predictions.shape

(4058, 1)

In [0]:
norm_predictions.shape

(4058, 1)

In [0]:
norm_predictions

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [0]:
import pandas as pd


In [0]:
sub_df = pd.read_csv('/content/random_submission.csv')

In [0]:
sub_df['Prediction'] = predictions

In [0]:
sub_df

,Id,Prediction
0,17000,0
1,17001,0
2,17002,0
3,17003,0
4,17004,0
...,...,...
4053,23585,0
4054,23586,0
4055,23587,0
4056,23588,0


In [0]:
sub_df.to_csv('/content/wo_norm_submission.csv', index=False)

In [0]:
norm_sub_df = sub_df.copy()

In [0]:
norm_sub_df['Prediction'] = norm_predictions

In [0]:
norm_sub_df.to_csv('/content/norm_submission.csv', index=False)


In [0]:
!kaggle competitions submit -c decoding-the-human-brain -f /content/wo_norm_submission.csv -m "w/o norm submission"

100% 31.7k/31.7k [00:08<00:00, 3.74kB/s]
Successfully submitted to DecMeg2014 - Decoding the Human Brain

In [0]:
!kaggle competitions submissions -c decoding-the-human-brain 

fileName                date                 description          status    publicScore  privateScore  
----------------------  -------------------  -------------------  --------  -----------  ------------  
wo_norm_submission.csv  2020-04-12 23:48:31  w/o norm submission  complete  0.50000      0.50000       
wo_norm_submission.csv  2020-04-12 23:27:43  w/o norm submission  complete  0.50000      0.50000       
norm_submission.csv     2020-04-12 23:05:36  norm submission      complete  0.50623      0.53661       
wo_norm_submission.csv  2020-04-12 22:49:45  w/o norm submission  complete  0.49943      0.53225       
norm_submission.csv     2020-04-12 22:20:16  norm submission      complete  0.51360      0.50305       
wo_norm_submission.csv  2020-04-12 22:19:46  w/o norm submission  complete  0.49433      0.50959       
norm_submission.csv     2020-04-12 18:30:42  norm submission      complete  0.50680      0.51394       
wo_norm_submission.csv  2020-04-12 18:27:32  w/o norm submission

In [0]:
!kaggle competitions submit -c decoding-the-human-brain -f /content/norm_submission.csv -m "norm submission"

100% 31.7k/31.7k [00:05<00:00, 5.57kB/s]
Successfully submitted to DecMeg2014 - Decoding the Human Brain

In [0]:
!kaggle competitions submissions -c decoding-the-human-brain 


fileName                date                 description          status    publicScore  privateScore  
----------------------  -------------------  -------------------  --------  -----------  ------------  
norm_submission.csv     2020-04-12 23:52:57  norm submission      complete  0.53798      0.52920       
wo_norm_submission.csv  2020-04-12 23:48:31  w/o norm submission  complete  0.50000      0.50000       
wo_norm_submission.csv  2020-04-12 23:27:43  w/o norm submission  complete  0.50000      0.50000       
norm_submission.csv     2020-04-12 23:05:36  norm submission      complete  0.50623      0.53661       
wo_norm_submission.csv  2020-04-12 22:49:45  w/o norm submission  complete  0.49943      0.53225       
norm_submission.csv     2020-04-12 22:20:16  norm submission      complete  0.51360      0.50305       
wo_norm_submission.csv  2020-04-12 22:19:46  w/o norm submission  complete  0.49433      0.50959       
norm_submission.csv     2020-04-12 18:30:42  norm submission    